In [2]:
import pandas as pd
from Bio import SeqIO
from Bio.ExPASy import ScanProsite

So I dun goofed and forgot to have you you guys download sequences as well for this next step, below is a script you help you merge the two files automatically, you'll use a similar pattern for the next set of steps which invovles extracting protien domains. 

In [3]:
#Read your blast results csv in
df = pd.read_csv("C:\\Users\\cmgro\\Downloads\\yhiD_genome_feature (12).csv")
#Show the first 5 rows
print(df.head())
#Temp lists to attatch variables 
ids = []
seqs = []
#Seq.IO.parse is. an easy way to read through every gene/prot in a fasta file 
for record in SeqIO.parse("C:\\Users\\cmgro\\Downloads\\yhiD_genome_feature (12).fasta", "fasta"):
    # So the fasta ID line is fig|1240239.3.peg.2328|XFF4834R_chr22840|VBIXanFus78442_2328|
    # but we only need the first two elements ("fig and 1240239.3.peg.2328") to make fig|1240239.3.peg.2328
    # We then use this ID to merge back onto the original dataframe (basically we say if this row has this ID, add the sequence to that row)
    
    #Fancy way of extracting the ID
    ids.append("|".join(record.id.split("|")[0:2]))
    #Fancy way of extracting the sequence 
    seqs.append(str(record.seq))
#making a new dataframe from this info 
seq_df = pd.DataFrame.from_dict({"PATRIC ID": ids, "Sequences":seqs})
print(seq_df.head())
#Mergine back and saving 
df = df.merge(seq_df, on="PATRIC ID")
df.to_csv("/Users/cmgro/Downloads/merged_yhiD.csv")

                                 Genome  Genome ID        Accession  \
0        Prevotella stercorea DSM 18206  1002367.3     AFZZ01000136   
1               Paenibacillus elgii B69  1007103.3     AFHW01000170   
2                 Bacillus sp. 10403023  1033734.3     CAET01000080   
3     Lactobacillus kefiranofaciens ZW3  1033837.3        NC_015602   
4  Brevibacillus laterosporus LMG 15441  1042163.3  NZ_AFRV01000008   

                PATRIC ID  RefSeq Locus Tag         Alt Locus Tag  \
0  fig|1002367.3.peg.1265  HMPREF0673_01564  VBIPreSte187147_1265   
1  fig|1007103.3.peg.6796               NaN  VBIPaeElg187263_6796   
2  fig|1033734.3.peg.4466               NaN   VBIBacSp210989_4466   
3  fig|1033837.3.peg.1858         WANG_1450  VBILacKef189943_1858   
4  fig|1042163.3.peg.4641       BRLA_c48920  VBIBreLat201509_4641   

                                          Feature ID Annotation Feature Type  \
0  PATRIC.1002367.3.AFZZ01000136.CDS.14973.15632.rev     PATRIC          CDS  

In [ ]:
import time
sigs = []

for i in df["Sequences"].values:
    handle = ScanProsite.scan(seq=i, mirror="https://prosite.expasy.org")
    result = ScanProsite.read(handle)
    qq = []
    for i in result:
        print(i)
        qq.append(i["signature_ac"])
    time.sleep(5)
    qq = set(qq)
    sigs.append(qq)
    
df["Prosite results"] = sigs
df.to_csv("/Users/christine/Downloads/yhiD_motifs.csv")

In [ ]:
df
